In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Set the API key for Generative AI
genai.configure(api_key=os.getenv("GENAI_API_KEY"))

# Define the prompt for the model
prompt = """Given a dataset with following properties:
        - Number of samples: 1000 images
        - Image size: 320x240 pixels
        - Task type: classification
        - Number of classes: 8

        Suggest an optimal neural network architecture for this classification task.  Provide the response as a *pure*, valid JSON string with *no* surrounding text or explanations. Do start with "{" and do not use newline codes or tabs. The JSON should have a "layers" field, an "optimizer" field, a "learning_rate" field, a "loss" field, a "metrics" field, a "batch_size" field, and an "epochs" field.
        """

# Call the Generative AI model with the prompt
model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

In [2]:
res = model.generate_content(prompt)

In [3]:
res.text

'```json\n{\n  "layers": [\n    {"type": "Conv2D", "filters": 32, "kernel_size": [3, 3], "activation": "relu", "input_shape": [320, 240, 3]},\n    {"type": "MaxPooling2D", "pool_size": [2, 2]},\n    {"type": "Conv2D", "filters": 64, "kernel_size": [3, 3], "activation": "relu"},\n    {"type": "MaxPooling2D", "pool_size": [2, 2]},\n    {"type": "Conv2D", "filters": 128, "kernel_size": [3, 3], "activation": "relu"},\n    {"type": "MaxPooling2D", "pool_size": [2, 2]},\n    {"type": "Flatten"},\n    {"type": "Dense", "units": 128, "activation": "relu"},\n    {"type": "Dropout", "rate": 0.5},\n    {"type": "Dense", "units": 8, "activation": "softmax"}\n  ],\n  "optimizer": "Adam",\n  "learning_rate": 0.001,\n  "loss": "categorical_crossentropy",\n  "metrics": ["accuracy"],\n  "batch_size": 32,\n  "epochs": 30\n}\n```'

In [4]:
import json
result = res.text.strip()
result = result.replace("\n", "").replace("\t", "")
result = result.replace("```json\n", "").replace("```", "").replace("json{", "{").replace("}", "}")
print(result)
response = json.loads(result)
print(response)


{  "layers": [    {"type": "Conv2D", "filters": 32, "kernel_size": [3, 3], "activation": "relu", "input_shape": [320, 240, 3]},    {"type": "MaxPooling2D", "pool_size": [2, 2]},    {"type": "Conv2D", "filters": 64, "kernel_size": [3, 3], "activation": "relu"},    {"type": "MaxPooling2D", "pool_size": [2, 2]},    {"type": "Conv2D", "filters": 128, "kernel_size": [3, 3], "activation": "relu"},    {"type": "MaxPooling2D", "pool_size": [2, 2]},    {"type": "Flatten"},    {"type": "Dense", "units": 128, "activation": "relu"},    {"type": "Dropout", "rate": 0.5},    {"type": "Dense", "units": 8, "activation": "softmax"}  ],  "optimizer": "Adam",  "learning_rate": 0.001,  "loss": "categorical_crossentropy",  "metrics": ["accuracy"],  "batch_size": 32,  "epochs": 30}
{'layers': [{'type': 'Conv2D', 'filters': 32, 'kernel_size': [3, 3], 'activation': 'relu', 'input_shape': [320, 240, 3]}, {'type': 'MaxPooling2D', 'pool_size': [2, 2]}, {'type': 'Conv2D', 'filters': 64, 'kernel_size': [3, 3], 'act

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd

# Create a random classification dataset
X, y = make_classification(n_samples=5000, n_features=20, n_classes=4, n_informative=8, shuffle=True)

print("Dataset created with shape:")
# Save the dataset into a CSV file
df = pd.DataFrame(X, columns=[f"Feature_{i}" for i in range(X.shape[1])])
df['Label'] = y
df.to_csv("classification_dataset.csv", index=False)
print("Dataset saved to classification_dataset.csv")
print("Labels (y):", y.shape)

Dataset created with shape:
Dataset saved to classification_dataset.csv
Labels (y): (5000,)


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load and preprocess your data
def preprocess_data(df, target_column):
    # Split features and target
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Handle categorical features
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = LabelEncoder().fit_transform(X[col])

    # Handle categorical target
    if y.dtype == 'object' or y.dtype.name == 'category':
        le = LabelEncoder()
        y = le.fit_transform(y)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y, scaler

# Build model with your custom architecture
def build_model(input_dim=4):
    model = Sequential()
    model.add(Dense(3, activation='linear', input_shape=(input_dim,)))
    model.add(Dense(2, activation='linear'))
    model.add(Dense(1, activation='linear'))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam', loss=mse, metrics=['mae'])
    return model

# Train model
def train_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32):
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    return history

# Evaluate model
def evaluate_model(model, X_test, y_test):
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    return test_loss, test_acc

# Make predictions
def predict(model, X_new, scaler):
    # Scale new data using the same scaler
    X_new_scaled = scaler.transform(X_new)
    predictions = model.predict(X_new_scaled)
    return predictions

# Example usage
if __name__ == "__main__":
    # Load data
    df = pd.read_csv('your_data.csv')

    # Preprocess
    X, y, scaler = preprocess_data(df, 'target_column')

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build and train model
    model = build_model(input_dim=X.shape[1])  # Pass input dimension dynamically
    history = train_model(model, X_train, y_train, X_test, y_test)

    # Evaluate
    test_loss, test_acc = evaluate_model(model, X_test, y_test)
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # Save model
    model.save('model.h5')
    
    # Quick model summary
    model.summary()
